In [7]:
import numpy as np
from PIL import Image
from ultralytics import YOLO

def predict_single_image(model, image_path, image_size=(224, 224)):
    """
    Predict the class of a single image using a YOLOv8-cls model.
    
    Args:
        model: A loaded YOLOv8-cls model
        image_path: String path to the image file
        image_size: Tuple of (width, height) for input image size, default (224, 224)
    """
    # Open and preprocess the image
    img = Image.open(image_path)
    
    if img.mode != 'RGB':
        img = img.convert('RGB')
    
    # Resize the image - note that PIL takes (width, height)
    img = img.resize(image_size, Image.Resampling.LANCZOS)
    
    # Convert to numpy array and ensure correct dimensions
    img_array = np.array(img)
    
    # Ensure the image has the right shape (height, width, channels)
    print(f"Image shape before processing: {img_array.shape}")
    
    # Normalize pixel values
    img_array = img_array.astype(np.float32) / 255.0
    
    # Make prediction
    results = model.predict(source=image_path, verbose=False)
    
    # Process results
    # Get the probabilities from the results
    probs = results[0].probs
    predicted_class = int(probs.top1)
    confidence = float(probs.top1conf)
    
    return {
        'predicted_class': predicted_class,
        'confidence': confidence,
        'all_probabilities': probs.data.tolist()
    }

In [9]:
# Load your model
model = YOLO(r'C:\Users\kanan\Desktop\Project_TMJOA\2D_Pipeline\YOLO\runs\classify\train\weights\best.pt')  # Replace with your model path

# Path to your image
image_path = r'D:\Kananat\TF_TMJOA_jpg_x_5px_test_batch_by_ID\erosion_0\51-3282 R\51-3282 R_x_088.jpg'  # Replace with your image path

# Get prediction
result = predict_single_image(model, image_path)

# Print results in a readable format
print(f"Predicted class: {result['predicted_class']}")
print(f"Confidence: {result['confidence']:.2%}")
print(f"all_probabilities: {result['all_probabilities']}")

Image shape before processing: (224, 224, 3)
Predicted class: 0
Confidence: 99.77%
all_probabilities: [0.997739315032959, 0.002260662382468581]


In [ ]:
import os
from pathlib import Path

In [ ]:
def predict_folder_images(model, folder_path):
    """
    Predict classes for all images in a given folder using a YOLOv8-cls model.
    
    Args:
        model: A loaded YOLOv8-cls model
        folder_path: String or Path object pointing to the folder containing images
    
    Returns:
        Dictionary where:
            - keys are image filenames
            - values are prediction dictionaries containing:
                - predicted_class
                - confidence
                - all_probabilities
    """
    # Convert folder_path to Path object for better path handling
    folder_path = Path(folder_path)
    
    # Dictionary to store results for each image
    predictions = {}
    
    # List of common image extensions we want to process
    # We can easily add more extensions if needed
    image_extensions = ('.jpg', '.jpeg', '.png', '.bmp', '.tif', '.tiff')
    
    try:
        # Iterate through all files in the folder
        for file_path in folder_path.iterdir():
            # Check if the file is an image by looking at its extension
            if file_path.suffix.lower() in image_extensions:
                try:
                    # Get predictions for this image
                    result = predict_single_image(model, str(file_path))
                    
                    # Store the result with the filename as key
                    predictions[file_path.name] = result
                    
                    # Print progress (optional, but helpful for monitoring)
                    print(f"Processed {file_path.name}: Class {result['predicted_class']} "
                          f"with confidence {result['confidence']:.2%}")
                          
                except Exception as e:
                    # If there's an error with one image, log it but continue processing others
                    print(f"Error processing {file_path.name}: {str(e)}")
                    predictions[file_path.name] = {"error": str(e)}
        
        return predictions
    
    except Exception as e:
        # Handle any errors that might occur when accessing the folder
        raise Exception(f"Error accessing folder {folder_path}: {str(e)}")


In [ ]:
# Set up paths
images_folder = r'D:\Kananat\TF_TMJOA_jpg_x_5px_test_batch_by_ID\erosion_0\51-3282 R'  # Replace with your folder path

# Load the model
model = YOLO(r'C:\Users\kanan\Desktop\Project_TMJOA\2D_Pipeline\YOLO\runs\classify\train\weights\best.pt')  # Replace with your model path

# Process all images
results = predict_folder_images(model, images_folder)

# Print a summary of the results
print("\nPrediction Summary:")
print("-" * 50)
for image_name, prediction in results.items():
    if "error" in prediction:
        print(f"{image_name}: Error - {prediction['error']}")
    else:
        print(f"{image_name}: Class {prediction['predicted_class']} "
                f"(Confidence: {prediction['confidence']:.2%})")